In [33]:
from scipy.integrate import solve_ivp
import numpy as np
from scipy.integrate._ivp.ivp import OdeResult

### ODE solvers in Python HEM
The Python code uses [solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).

They are using two different *methods*:

- **RK45** - Runge-Kutta method of order 5(4), which is the default method
- **BDF** - Backward Differentiation Formula


The aim of this investigaion is the use a Rust library to replicate the same behaviour.
Example code below of usage.

Next steps:
- Investigate Rust libraries (ongoing)
- See how the results are used in Python. Do we need to match the exact method?

In [34]:
# example from docs
def exponential_decay(t, y): return -0.5 * y # an example function

t_span = [0, 5]
y0 = [2, 4, 8]

sol = solve_ivp(exponential_decay, t_span, y0, method="BDF")

print("\n\nBDF sol.t", sol.t)
print("\nBDF sol.y", sol.y)
print("\n\nBDF sol.y[:, -1]", sol.y[:, -1])

sol = solve_ivp(exponential_decay, t_span, y0) # RK45 by default

print("\n\nRK45 sol.t", sol.t)
print("\nRK45 sol.y", sol.y)
# print("RK45 sol.t_events[0][-1]", sol.t_events[0][-1]) # this currently errors
print("\n\nRK45 y[0][-1]", sol.y[0][-1]) 



BDF sol.t [0.00000000e+00 4.47278801e-03 8.94557602e-03 5.36734561e-02
 9.84013363e-02 2.75264786e-01 4.52128236e-01 6.28991687e-01
 1.00216860e+00 1.37534551e+00 1.74852243e+00 2.12169934e+00
 2.62806710e+00 3.13443486e+00 3.64080261e+00 4.14717037e+00
 4.65353813e+00 5.00000000e+00]

BDF sol.y [[2.         1.99553564 1.991081   1.94737462 1.90460653 1.74371599
  1.59604552 1.46079117 1.21192533 1.00575249 0.83476754 0.69282111
  0.53782659 0.41739566 0.32394847 0.25146823 0.19521498 0.16416546]
 [4.         3.99107127 3.98216199 3.89474925 3.80921307 3.48743198
  3.19209103 2.92158233 2.42385066 2.01150498 1.66953509 1.38564222
  1.07565318 0.83479132 0.64789694 0.50293646 0.39042995 0.32833091]
 [8.         7.98214255 7.96432399 7.78949849 7.61842613 6.97486396
  6.38418207 5.84316466 4.84770131 4.02300995 3.33907017 2.77128443
  2.15130635 1.66958265 1.29579388 1.00587291 0.7808599  0.65666183]]


BDF sol.y[:, -1] [0.16416546 0.32833091 0.65666183]


RK45 sol.t [0.         0.1148

### ODE solvers in Rust

#### [ode_solvers](https://crates.io/crates/ode_solvers)

Supports RK4 (Runge Katta 4) - might be okay, but slightly different to RK45.

Does not support BDF.

##### [fast_ode](https://crates.io/crates/fast_ode)

Supports RK45, intended to be identical to scipy's implementation.

Does not support BDF.

### Usage In Python HEM

The code below shows where ODE solvers are used in Python HEM.
NOTE - these currently won't run in this notebook!

In [35]:
# from elec_storage_heater.py (modifed)
sol: OdeResult = solve_ivp(fun=self.__func_core_temperature_change_rate(q_dis_modo=q_dis_modo),
            t_span=time_range,
            y0=temp_core_and_wall,
            method='BDF')

new_temp_core_and_wall: list = sol.y[:, -1]

# uses BDF method - "Backward Differentiation Formula"

NameError: name 'self' is not defined

In [6]:
# from emitters.py
temp_diff_emitter_rm_results = solve_ivp(
            func_temp_emitter_change_rate,
            (time_start, time_end),
            (temp_diff_start,),
            events=events,
            )

# later in the function (modified)
#temp_diff_emitter_rm_results.t_events[0][-1] and temp_diff_emitter_rm_results.y[0][-1] are used


time_temp_diff_max_reached = temp_diff_emitter_rm_results.t_events[0][-1]
temp_diff_emitter_rm_final = temp_diff_emitter_rm_results.y[0][-1]

# uses default method, which is RK45 (Runge-Kutta method of order 5(4))

NameError: name 'func_temp_emitter_change_rate' is not defined